### Madina Hayva Putri - 5220411187
### Pemrosesan Teks - B

# Scraping Komentar Youtube Berdasarkan Kata Kunci

In [19]:
!pip install google-api-python-client
!pip install tqdm

In [20]:
from google.colab import userdata
from googleapiclient.discovery import build
import pandas as pd
from tqdm import tqdm

In [29]:
api_key = userdata.get('YouTube')  # ganti nama secret dengan nama yang sama di tab "Rahasia"
youtube = build('youtube', 'v3', developerKey=api_key)

print("Berhasil")

Berhasil


In [40]:
# parameter pencarian video dengan kata kunci
query = "review makanan tiktokers"
max_komen_per_video = 150
min_total_komen = 1000

In [41]:
videos = []
next_page_token = None

print("Mengambil daftar video berdasarkan kata kunci...")
while len(videos) < (min_total_komen // max_komen_per_video) + 5:
    request = youtube.search().list(
        q=query,
        part="id,snippet",
        type="video",
        maxResults=30,
        pageToken=next_page_token
    )
    response = request.execute()
    videos += response["items"]
    next_page_token = response.get("nextPageToken")
    if not next_page_token:
        break

print(f"Ditemukan {len(videos)} video terkait '{query}'\n")


Mengambil daftar video berdasarkan kata kunci...
Ditemukan 30 video terkait 'review makanan tiktokers'



In [45]:
data_komen = []
total_komen = 0

for video in tqdm(videos, desc="Mengambil komentar"):
    video_id = video["id"]["videoId"]
    channel_name = video["snippet"]["channelTitle"]
    video_title = video["snippet"]["title"]

    next_page_token = None
    count = 0

    while count < max_komen_per_video:
        request = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,
            pageToken=next_page_token,
            textFormat="plainText"
        )
        response = request.execute()

        for item in response["items"]:
            comment = item["snippet"]["topLevelComment"]["snippet"]
            data_komen.append({
                "nama_channel": channel_name,
                "judul_video": video_title,
                "komentar": comment["textDisplay"],
                "jumlah_like_komentar": comment["likeCount"],
                "tanggal_posting": comment["publishedAt"]
            })
            count += 1
            total_komen += 1
            if count >= max_komen_per_video:
                break  # hanya berhenti per video

        next_page_token = response.get("nextPageToken")
        if not next_page_token:
            break  # komentar video habis

print(f"\nTotal komentar terkumpul: {len(data_komen)}")

Mengambil komentar: 100%|██████████| 30/30 [00:05<00:00,  5.20it/s]


Total komentar terkumpul: 4069


## Casefolding

In [55]:
import re
df = pd.DataFrame(data_komen)
df["komentar"] = df["komentar"].apply(lambda x: x.casefold()) #mengubah huruf menjadi kecil
df["komentar"] = df["komentar"].str.strip() #menghapus spasi berlebih
df["komentar"] = df["komentar"].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x)) #menghapus karakter non-alfabet

print(f"Total komentar setelah casefolding: {len(df)}")

Total komentar setelah casefolding: 4069


In [56]:
df.head(10)

,nama_channel,judul_video,komentar,jumlah_like_komentar,tanggal_posting
0,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,selalu ngiler,0,2025-09-29T08:15:18Z
1,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,cookies minbite,0,2025-09-06T16:56:33Z
2,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,kak bikin review gini lagiiiiiiikiii sukak ban...,0,2025-09-02T09:38:41Z
3,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,ngoler tahu sama basreng nya,0,2025-08-28T02:17:59Z
4,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,salam kenal dari tabanan bali semoga sehat sel...,0,2025-07-22T14:51:17Z
5,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,jando itu lemak tapi bagian dari pd nya sapi b...,0,2025-06-01T15:58:14Z
6,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,kalau berani review nastar saya umma delight n...,0,2025-04-22T12:23:59Z
7,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,ada yg tau ga hijab yg dipake tasyi apa,1,2025-04-10T16:34:59Z
8,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,38r,0,2025-03-04T09:53:50Z
9,Tasyi Athasyia,TASYI REVIEW MAKANAN TIKTOKERS VIRAL😱⁉️ MAAF G...,kayanya enak ya,0,2025-02-26T02:41:13Z


In [57]:
df.to_csv("data_komen_youtube.csv", index=False, encoding="utf-8-sig")

print("File tersimpan sebagai 'data_komen_youtube.csv'")

File tersimpan sebagai 'data_komen_youtube.csv'
